<h1 style="text-align: center; color: purple;" markdown="1">Econ 320 Python Assignment 5  </h1>

<h2 style="text-align: center; color: #012169" markdown="1">Regressions with Qualitative Information </h2>

<h2 style="text-align: center; color: #012169" markdown="1">Zhou Fang </h2>

## Package setup

In [1]:
# import the necessary packages 

import scipy.stats as stats
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
import wooldridge as woo
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

## 1. Using the data wage2 from the wooldridge package. Estimate the models: m1, m2, m3, m4 

Model 1 $$log(wage) = \beta_0 + \beta_1 educ + \beta_2 exper + \beta_3tenure + \beta_4 married + u $$
Model 2  $$log(wage) = \beta_0 + \beta_1 educ + \beta_2 exper + \beta_3tenure + \beta_4 married + \beta_5 black + u $$
Model 3  $$log(wage) = \beta_0 + \beta_1 educ + \beta_2 exper + \beta_3tenure + \beta_4 married + \beta_5 black + \beta_6 south + u $$ 
Model 4  $$log(wage) = \beta_0 + \beta_1 educ + \beta_2 exper + \beta_3tenure + \beta_4 married + \beta_5 black + \beta_6 south + \beta_7 urban + u$$ 

Report the results using a stargazer table. Holding other factors fixed, what is the aproximate difference in monthly salary between blacks and non blacks, married vs single, urban vs rural?  Are these statistically significant? INTERPRET.

In [2]:
#load the data
wage2 =woo.dataWoo('wage2')

# set each regression
m1 = smf.ols(formula='np.log(wage) ~ educ + exper + tenure + married', data=wage2)
m1 = m1.fit()
m2 = smf.ols(formula='np.log(wage) ~ educ + exper + tenure + married + black', data=wage2)
m2 = m2.fit()
m3 = smf.ols(formula='np.log(wage) ~ educ + exper + tenure + married + black + south', data=wage2)
m3 = m3.fit()
m4 = smf.ols(formula='np.log(wage) ~ educ + exper + tenure + married + black + south + urban', data=wage2)
m4 = m4.fit()

In [3]:
#view the results using a stargazer table

st=Stargazer([m1, m2,m3,m4])
from IPython.core.display import HTML
HTML(st.render_html())

## 2. Using model 4, add the variables $exper^2$ and $tenure^2$ to the equation show the regression results and test if they are jointly significant at 20% level. What about 25% Level? Explain.

In [4]:

m5 = smf.ols(formula='np.log(wage) ~ educ + exper + I(exper**2) + tenure + I(tenure**2) + married + black + south + urban', data=wage2)
m5 = m5.fit()
m5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           np.log(wage)   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.248
Method:                 Least Squares   F-statistic:                     35.17
Date:                Wed, 02 Dec 2020   Prob (F-statistic):           1.22e-53
Time:                        12:47:22   Log-Likelihood:                -380.05
No. Observations:                 935   AIC:                             780.1
Df Residuals:                     925   BIC:                             828.5
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          5.3587      0.126     42.558      0.000       5.112       5.606
educ               0.0643      0.006     10.184      0.000       0.052       0.077
exper              0.0172      0.013      1.365      0.173      -0.008       0.042
I(exper ** 2)     -0.0001      0.001     -0.214      0.831      -0.001       0.001
tenure             0.0249      0.008      3.066      0.002       0.009       0.041
I(tenure ** 2)    -0.0008      0.000     -1.691      0.091      -0.002       0.000
married            0.1985      0.039      5.077      0.000       0.122       0.275
black             -0.1907      0.038     -5.057      0.000      -0.265      -0.117
south             -0.0912      0.026     -3.477      0.001      -0.143      -0.040
urban              0.1854      0.027      6.878      0.000       0.133       0.238
==============================================================================
Omnibus:                       37.281   Durbin-Watson:                   1.819
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               84.322
Skew:                          -0.199   Prob(JB):                     4.90e-19
Kurtosis:                       4.416   Cond. No.                     2.19e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.19e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [5]:
#vector with the names of the variables that you are testing 
hypotheses = ['I(exper ** 2)=0','I(tenure ** 2)=0']
#automated F test:

ftest = m5.f_test(hypotheses)
fstat = ftest.statistic[0][0]
fpval = ftest.pvalue

print(f'fstat: {fstat}\n')
print(f'fpval: {fpval}\n')

fstat: 1.4898059755281001

fpval: 0.22595675832786968



Since the p-value is 0.226. We fail to reject the null hypothesis at 20% level and conclude that the two variables are not jointly significant at 20% level. However, at 25% level, we reject H0 and conclude that the two variables are jointly significant at 25% level.

## 3. Using model 4 run model 6 by adding a dummy variable for age groups less than or equal to 30, between 30 to 35, older than 35. 
  - Do a frequency table using `pd.crosstab` showing the distribution of each group.  
  - Make older than 35 the reference group, run a regression, what can you say about the effect of age group on the wages? 
  - Show the regression results.
  - make an anova table for the F test

In [6]:
#3
ageg = [0, 30, 35, 38]
wage2['agegr'] = pd.cut(wage2['age'], bins = ageg,
                       labels=['Younger than 30', '30-34', '35 and up'])


# display frequencies:
freq = pd.crosstab(wage2['agegr'], columns='count')
print(f'freq: \n{freq}\n')

# run regression:
m6 = smf.ols(formula='np.log(wage) ~ C(agegr, Treatment("35 and up")) +'
              'educ + exper + tenure + married + black + south + urban',
              data=wage2)
m6 = m6.fit()
# print regression table:
m6.summary()

freq: 
col_0            count
agegr                 
Younger than 30    251
30-34              408
35 and up          276



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           np.log(wage)   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     35.42
Date:                Wed, 02 Dec 2020   Prob (F-statistic):           5.39e-54
Time:                        12:47:22   Log-Likelihood:                -379.20
No. Observations:                 935   AIC:                             778.4
Df Residuals:                     925   BIC:                             826.8
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=======================================================================================================================
                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
Intercept                                               5.5328      0.130     42.577      0.000       5.278       5.788
C(agegr, Treatment("35 and up"))[T.Younger than 30]    -0.0803      0.037     -2.157      0.031      -0.153      -0.007
C(agegr, Treatment("35 and up"))[T.30-34]              -0.0389      0.030     -1.278      0.202      -0.099       0.021
educ                                                    0.0619      0.006      9.585      0.000       0.049       0.075
exper                                                   0.0105      0.004      2.914      0.004       0.003       0.018
tenure                                                  0.0110      0.002      4.423      0.000       0.006       0.016
married                                                 0.1951      0.039      4.994      0.000       0.118       0.272
black                                                  -0.1883      0.038     -5.006      0.000      -0.262      -0.114
south                                                  -0.0918      0.026     -3.500      0.000      -0.143      -0.040
urban                                                   0.1835      0.027      6.807      0.000       0.131       0.236
==============================================================================
Omnibus:                       38.035   Durbin-Watson:                   1.835
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               81.703
Skew:                          -0.228   Prob(JB):                     1.81e-18
Kurtosis:                       4.374   Cond. No.                         217.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
# ANOVA table:
table_anova = sm.stats.anova_lm(m6, typ=2)
print(f'table_anova: \n{table_anova}\n')

table_anova: 
                                      sum_sq     df          F        PR(>F)
C(agegr, Treatment("35 and up"))    0.619655    2.0   2.326242  9.823319e-02
educ                               12.237293    1.0  91.879866  8.213351e-21
exper                               1.130583    1.0   8.488627  3.659777e-03
tenure                              2.604983    1.0  19.558694  1.091421e-05
married                             3.321320    1.0  24.937089  7.078633e-07
black                               3.337191    1.0  25.056252  6.665246e-07
south                               1.631948    1.0  12.252966  4.866986e-04
urban                               6.172039    1.0  46.340813  1.785375e-11
Residual                          123.198870  925.0        NaN           NaN



## 4. Use model 4 but now allow wages to differ across four groups of people, married and black, married and nonblack, single and black and single and non black. Put the results in a stargarzer table. *(Hint: Think about how interactions work, what are all the categories in the interaction and what is the base category.) Do not create the interaction separately. Do it inside the `ols()` directly as learned in class. Interpret your result
<br>

In [8]:
#4

m7 = smf.ols(formula = 'np.log(wage) ~ educ+exper+ tenure + south + urban + married+ black + married*black', 
             data = wage2).fit()
m7.summary()
#view the results using a stargazer table

st=Stargazer([m7])
st.covariate_order(['educ' , 'exper' , 'tenure', 'south', 'urban', 
                    'married' , 'black', "married:black","Intercept"])
from IPython.core.display import HTML
HTML(st.render_html())


The results indicate that married black people are more competitive than single black people. However, married black people are less competitive than married non-black people. 



<br>

**Notice that**
 
$\beta_{married}+\beta_{black}+\beta_{married\&black} =$ *Effect of being married and black vs single and non-black.*

0.189+ -0.241+ 0.061 = 0.009 Same number for 𝛽𝑚𝑎𝑟𝑟𝑖𝑒𝑑&𝑏𝑙𝑎𝑐𝑘 

<style>
div.gray { background-color:#dbdbdb; border-radius: 5px; padding: 20px;}
</style>
<div class = "gray">

**Packages used in this document**

`wooldridge`
`matplot.lib`
`pandas`
`numpy` 
`seaborn`
    

</div>

&nbsp;
<hr />
<p style="font-family:palatino; text-align: center;font-size: 15px">ECON220 Python Programming Laboratory</a></p>
<p style="font-family:palatino; text-align: center;font-size: 15px">Professor <em> Paloma Lopez de mesa Moyano</em></a></p>
<p style="font-family:palatino; text-align: center;font-size: 15px"><span style="color: #6666FF;"><em>paloma.moyano@emory.edu</em></span></p>

<p style="font-family:palatino; text-align: center;font-size: 15px">Department of Economics</a></p>
<p style="font-family:palatino; text-align: center; color: #012169;font-size: 15px">Emory University</a></p>

&nbsp;

In [9]:
!jupyter nbconvert --to html Econ220Lab_Assign5_QualitativeVars.ipynb

[NbConvertApp] WARNING | pattern 'Econ220Lab_Assign5_QualitativeVars.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--debug
    set log level to logging.DEBUG (maximize logging output)
--generate-config
    generate default config file
-y
    Answer yes to any questions instead of prompting.
--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--stdout
    Write n